# Import Scripts and Libraries

In [2]:
from signal_extraction import *
from mlutils import *
import os

print('Subjects:')

subject_names=os.listdir(BASE_EXTRACTION_PATH)
for s in subject_names:
    print('--> ',s)

Subjects:
-->  Abhoy
-->  Alok
-->  Tushar


In [7]:
import matplotlib.pyplot as plt
t, angle, emg = read_extracted_combined_data('Abhoy', 2)

%matplotlib qt

sl=slice(int(12.5*2000),int(12.8*2000))
plt.subplot(4,1,1)
plt.plot(t[sl], angle[sl])
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Angle')

muscles=['Biceps Brachii', 'Triceps Lateral', 'Triceps Medial']

for i in range(3):
    plt.subplot(4,1,2+i)
    plt.plot(t[sl], emg[i][sl])
    plt.grid(True)
    plt.xlabel('Time')
    plt.ylabel(muscles[i])




In [13]:
img=np.array(emg[:3])[:3,int(12.5*2000):int(12.8*2000)]
img=np.repeat(img,50,0)
plt.imshow(img, cmap='gray', vmax=1, vmin=0)
window=0.15



# Generating Classifier Signal

In [30]:
flex_ex=generate_classifier_signal(angle)


plt.plot(t,angle*np.pi/180,t,flex_ex)
plt.xlabel('Time')
plt.legend(['Angle (rad)','Class'])


In [6]:
muscles=['Biceps Brachii', 'Triceps Lateral', 'Triceps Medial']

plt.subplot(4,1,1)
plt.plot(t,angle*np.pi/180,t,flex_ex)
plt.xlabel('Time')
plt.legend(['Angle (rad)','Class'])

for i in range(3):
    plt.subplot(4,1,2+i)
    plt.plot(t, emg[i])
    plt.grid(True)
    plt.xlabel('Time')
    plt.ylabel('Activation RMS')
    plt.legend([muscles[i]])

In [65]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Example sequences
fs = 2000  # Sampling frequency in Hz
t = t
s1 = emg[0]
s2 = emg[1]
s3=emg[2]
s4=flex_ex
# s3 = np.sin(2 * np.pi * t) * np.cos(2 * np.pi * t)

time_step = 1 / fs  # Time between frames

fig, axs = plt.subplots(4, 1, figsize=(10, 7))
fig.waitforbuttonpress()
# fig.tight_layout()
xlabels=['Biceps Brachii', 'Triceps Lateral', 'Triceps Medial','Classifier']
lines = []
lims=((0,0.2),(0,0.5),(0,0.3),(-0.3,3.5))
cols=['b','b','b','orange']
def init():
    for ax,xl,lim,col in zip(axs,xlabels,lims,cols):
        line, = ax.plot([], [], col)
        ax.set_xlim(0, len(t)/10)
        ax.set_ylim(lim)
        ax.set_ylabel(xl)
        lines.append(line)
    return lines

def update(frame):
    for i, data in enumerate([s1[::10], s2[::10],s3[::10],s4[::10]]):
        lines[i].set_data(np.arange(frame+1), data[:frame+1])
    return lines

ani = animation.FuncAnimation(fig, update, frames=len(t), init_func=init, interval=time_step*1000, blit=True)
plt.show()


In [49]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Example sequences
fs = 2000  # Sampling frequency in Hz
t = t
s1 = angle * np.pi/180
s2 = flex_ex

time_step = 1 / fs  # Time between frames

fig, ax = plt.subplots(figsize=(15, 5))
fig.waitforbuttonpress()
lines = []
colors = ['b', 'orange']
labels = ['Angle', 'Classifier']

def init():
    for color, label in zip(colors, labels):
        line, = ax.plot([], [],color,label=label)
        lines.append(line)
    ax.set_xlim(0, len(t)/10)
    ax.set_ylim(-0.5, 3.5)
    ax.set_xlabel('Samples')
    ax.set_ylabel('Angle(in rads)/Class (0,1,2,3)')
    ax.legend()
    return lines

def update(frame):
    for i, data in enumerate([s1[::10], s2[::10]]):
        lines[i].set_data(np.arange(frame+1), data[:frame+1])
    return lines

ani = animation.FuncAnimation(fig, update, frames=len(t), init_func=init, interval=time_step * 1000, blit=True)
plt.show()
